In [1]:
'''
## Overview
This notebook summarizes the analysis for the ring-attractor networks
with cosine or von Mises weight functions and threshold-linear or tanh 
activation functions.

### Part I: Precise theoretical predictions vs simulation results
The notebook examines the correlation between the (a) activity
difference between the right and left rings and (b) angular head velocity.

The notebook also examines other predictions: activity symmetry for the 
shifter-ring network, special case when W_s=W_d, the same activity shape
for the synaptic modulation netowrk.
See "Manuscript_Supplementary/supplementary.pdf" for details.


### Part II: Approximate theoretical prediction vs simulation results
The notebook saves scatter plots to see if the approximate theoretical 
predictions fit the simulation results.
The theoretical formula: right-left difference = slope * k omega, in the
derivation using first order approximation, slope depends on 
WD = Mean(W_s) - Mean(W_d), and when WD = 0, slope = 2. See 
"Manuscript_Supplementary/3. Shifter-ring network" for details. 


## Author
Siyuan Mei (mei@bio.lmu.de)

## Last update
2025-9-11: add docstring for the notebook
'''
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
from scipy.integrate import solve_ivp
from matplotlib import colors
from scipy.signal import find_peaks
from scipy.stats import pearsonr
import pandas as pd
import seaborn as sns

import HD_utils.circular_stats as cstat
from HD_utils.network import *
from HD_utils.matrix import *
from HD_utils.adap_sim_move import *
from HD_utils.adap_sim_stable import *
from HD_utils.IO import *
from HD_utils.plot import *
from HD_utils.comput_property import *
from HD_utils.exam import *
from HD_utils.gridsearch_configs import configs
import HD_utils.dataclass as dc

import functions_21 as funcs

pd.options.display.max_columns = 100

# Load

In [2]:
# # Load saved results: run the current notebook once before loading the results
# stat_1ring = pd.read_pickle(STAT_PATH / 'stat_1ring.npy')
# stat_2ring = pd.read_pickle(STAT_PATH / 'stat_2ring.npy')
# stat_3ring = pd.read_pickle(STAT_PATH / 'stat_3ring.npy')
# stat_shifter = pd.read_pickle(STAT_PATH / 'stat_shifter.npy')

# Synpase-modulation

In [3]:
stat_1ring = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: shape deviation $d_{rel}$', 'SD: shape deviation $d_{rel}$', 'Max: shape deviation $d_{rel}$', 'Mean: peak firing deviation $d_{frel}$', 'SD: peak firing deviation $d_{frel}$', 'Max: peak firing deviation $d_{frel}$'])
structure = 'Synapse-modulation'

dev_shape_ratios_list = []
dev_heightratios_list = []

for iorder, config in enumerate(configs[:4]): # 1 ring
    assert config.ring_num == 1
    
    net_stationary = load_gridsearch_results("stationary", config)
    net_moving = load_gridsearch_results("moving", config)
    net_moving.calculate_linear_range(net_stationary, config)
    
    # Examine theoretical predictions
    index_shape_mismatch, dev_shape_ratios, if_match = cal_shape_mismatch_loop(net_moving.activity, net_moving.linear_id, config.zeroid)
    index_height_mismatch, dev_heightratios, if_match = cal_height_dif_loop(net_moving.activity, net_moving.linear_id, config.actfun, config.zeroid, b=config.bs)

    dev_shape_ratios = dev_shape_ratios[net_moving.linear_id]
    dev_heightratios = dev_heightratios[net_moving.linear_id]
    dev_shape_ratios_list.append(dev_shape_ratios)
    dev_heightratios_list.append(dev_heightratios)
    # Result storage
    stat_1ring.loc[iorder] = [structure, config.ring_num, 
                              waf_df_names[config.weight_fun.__name__], 
                              waf_df_names[config.actfun.__name__], 
                              len(net_stationary.eval), net_stationary.valid_num, 
                              len(net_moving.linear_id), 
                              np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios), 
                              np.mean(dev_heightratios), np.std(dev_heightratios), np.max(dev_heightratios)]
    
dev_shape_ratio_all = np.concatenate(dev_shape_ratios_list)
dev_heightratio_all = np.concatenate(dev_heightratios_list)

stat_1ring.loc[iorder+1] = [structure, config.ring_num, 
                            'all', 
                            'all', 
                            stat_1ring['N'].sum(), 
                            stat_1ring['N: valid stationary'].sum(), 
                            stat_1ring['N: linearly integrates'].sum(),
                            np.mean(dev_shape_ratio_all), np.std(dev_shape_ratio_all), np.max(dev_shape_ratio_all), 
                            np.mean(dev_heightratio_all), np.std(dev_heightratio_all), np.max(dev_heightratio_all)]

stat_1ring.to_pickle(STAT_PATH / 'stat_1ring.npy')
stat_1ring.to_csv(TABLE_PATH / 'Supplementary table 1. Simulation results of synpase-modulation network.csv')

In [4]:
stat_1ring.iloc[:,1:]

,Ring No.,Weight Func,Act Func,N,N: valid stationary,N: linearly integrates,Mean: shape deviation $d_{rel}$,SD: shape deviation $d_{rel}$,Max: shape deviation $d_{rel}$,Mean: peak firing deviation $d_{frel}$,SD: peak firing deviation $d_{frel}$,Max: peak firing deviation $d_{frel}$
0,1,cos,max0x,676,366,366,0.119709,0.120894,0.480024,0.005539,0.005441,0.024125
1,1,cos,tanh,676,459,459,0.131736,0.124904,0.590559,0.001859,0.003677,0.019267
2,1,vonMises,max0x,4056,1845,1845,0.092380,0.111363,0.510432,0.007101,0.008331,0.051741
3,1,vonMises,tanh,4056,958,958,0.101967,0.111943,0.585946,0.000850,0.002546,0.021694
4,1,all,all,9464,3628,3628,0.102648,0.115127,0.590559,0.004630,0.007045,0.051741


# Shifter network

In [5]:
cor_array_list = [[], [], [], []]
rl_same_dev_list = []
rl_sym_dev_list = []
sym_sym_dev_list = []

## Two-ring

In [6]:
stat_2ring_eqw = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func equality', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: RL same $d_u-2k\omega$', 'SD: RL same $d_u-2k\omega$', 'Max: RL same $d_u-2k\omega$', 'Mean: RL Corr (A)', 'SD: RL Corr (A)', 'Min RL Corr (A)', 'Mean: RL Corr (B)', 'SD: RL Corr (B)', 'Min RL Corr (B)', 'Mean: RL Corr (C)', 'SD: RL Corr (C)', 'Min RL Corr (C)', 'Mean: RL Corr (D)', 'SD: RL Corr (D)', 'Min RL Corr (D)'])

stat_2ring_neqw = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func equality', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: RL Corr (A)', 'SD: RL Corr (A)', 'Min RL Corr (A)', 'Mean: RL Corr (B)', 'SD: RL Corr (B)', 'Min RL Corr (B)', 'Mean: RL Corr (C)', 'SD: RL Corr (C)', 'Min RL Corr (C)', 'Mean: RL Corr (D)', 'SD: RL Corr (D)', 'Min RL Corr (D)', 'Mean: RL Sym $d_{rel}$', 'SD: RL Sym $d_{rel}$', 'Max: RL Sym $d_{rel}$'])

structure = 'Shifter-ring (two rings)'
wfun_eqs = [True]*4 + [False]*4

for iorder, config in enumerate(tqdm(configs[4:12])):
    assert config.ring_num == 2
    
    wfun_eq = wfun_eqs[iorder]

    net_stationary = load_gridsearch_results("stationary", config)
    net_moving = load_gridsearch_results("moving", config)
    net_moving.calculate_linear_range(net_stationary, config)
    
    # Theoretical prediction examination
    bump_amplitudes = cal_firate_a_acv_mean_a_peak(
        net_moving.activity, 
        config.inputs, 
        net_moving.part_linear_id, 
        config.bs, config.actfun)
    
    input_diff_cors, input_diff_ps = cal_input_diff_cor(
        config.inputs, bump_amplitudes[4:], 
        net_moving.part_linear_id, net_moving.linear_range_id) 
    
    # calculate the mean, sd, min of the correlations
    show_value = input_diff_cors[net_moving.part_linear_id]
    cor_list = []
    for i in range(4):
        cor_list.append(np.nanmean(show_value[:,i]))
        cor_list.append(np.nanstd(show_value[:,i]))
        cor_list.append(np.nanmin(show_value[:,i]))
        
        cor_array_list[i].append(show_value[:,i])

    if wfun_eq: # If Weight fun equal

        # The sameness of the left and right rings
        index_shape_mismatch, dev_shape_ratios, if_match = cal_lr_shape_same_loop(net_moving.activity, net_stationary.valid_id, config.zeroid)
        dev_shape_ratios = dev_shape_ratios[net_stationary.valid_id]
        mean_same_dev, sd_same_dev, max_same_dev = np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios)
        rl_same_dev_list.append(dev_shape_ratios)
    
    else: # If Weight fun not equal
        # Mirror symmetry of the left and right rings
        index_shape_mismatch, dev_shape_ratios, if_match = cal_lr_shape_match_loop(net_moving.activity, net_moving.stable_id, config.zeroid)
        dev_shape_ratios = dev_shape_ratios[net_moving.stable_id]
        mean_match_dev, sd_match_dev, max_match_dev = np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios)
        rl_sym_dev_list.append(dev_shape_ratios)

    # Store the results
    total_num = len(net_stationary.eval)
    linear_num = len(net_moving.linear_id)
    if wfun_eq:
        stat_2ring_eqw.loc[iorder] = [structure, config.ring_num, wfun_eq, waf_df_names[config.weight_fun.__name__], waf_df_names[config.actfun.__name__],total_num, net_stationary.valid_num, linear_num, mean_same_dev, sd_same_dev, max_same_dev] + cor_list
    else:
        stat_2ring_neqw.loc[iorder] = [structure, config.ring_num, wfun_eq, waf_df_names[config.weight_fun.__name__], waf_df_names[config.actfun.__name__], total_num, net_stationary.valid_num, linear_num] + cor_list + [mean_match_dev, sd_match_dev, max_match_dev]

stat_2ring = pd.concat([stat_2ring_eqw, stat_2ring_neqw], ignore_index=True)

stat_2ring.to_pickle(STAT_PATH / 'stat_2ring.npy')
stat_2ring.to_csv(TABLE_PATH / 'Supplementary table 2. Simulation results of shifter-ring network with two rings.csv')

 12%|█▎        | 1/8 [00:02<00:15,  2.20s/it]c:\ProgramData\miniforge3\envs\HD\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
 38%|███▊      | 3/8 [00:15<00:31,  6.34s/it]c:\ProgramData\miniforge3\envs\HD\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
 62%|██████▎   | 5/8 [00:22<00:12,  4.25s/it]c:\ProgramData\miniforge3\envs\HD\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
 88%|████████▊ | 7/8 [00:34<00:05,  5.82s/it]c:\ProgramData\miniforge3\envs\HD\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the corre

In [7]:
stat_2ring.iloc[:,1:]

,Ring No.,Weight Func equality,Weight Func,Act Func,N,N: valid stationary,N: linearly integrates,Mean: RL same $d_u-2k\omega$,SD: RL same $d_u-2k\omega$,Max: RL same $d_u-2k\omega$,Mean: RL Corr (A),SD: RL Corr (A),Min RL Corr (A),Mean: RL Corr (B),SD: RL Corr (B),Min RL Corr (B),Mean: RL Corr (C),SD: RL Corr (C),Min RL Corr (C),Mean: RL Corr (D),SD: RL Corr (D),Min RL Corr (D),Mean: RL Sym $d_{rel}$,SD: RL Sym $d_{rel}$,Max: RL Sym $d_{rel}$
0,2,True,cos,max0x,676,367,28,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.088663e-17,1.000000,1.000000,3.079205e-17,1.000000,0.999723,0.000631,0.997693,0.998326,0.002082,0.992690,NaN,NaN,NaN
1,2,True,cos,tanh,676,471,94,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,7.368809e-18,1.000000,1.000000,3.456278e-17,1.000000,0.993257,0.009249,0.961215,0.993829,0.049334,0.323207,NaN,NaN,NaN
2,2,True,vonMises,max0x,4056,2049,138,6.911277e-16,4.200564e-15,2.165755e-13,1.000000,8.473586e-18,1.000000,1.000000,3.459327e-17,1.000000,0.999800,0.000724,0.989547,0.998557,0.001642,0.988717,NaN,NaN,NaN
3,2,True,vonMises,tanh,4056,1010,426,2.997150e-16,1.673214e-15,1.116367e-13,1.000000,1.626653e-17,1.000000,1.000000,5.333339e-17,1.000000,0.985143,0.014734,0.841815,0.998435,0.011557,0.727529,NaN,NaN,NaN
4,2,False,cos,max0x,1210,253,122,NaN,NaN,NaN,0.996136,1.107541e-02,0.917865,0.996136,1.107541e-02,0.917865,0.999819,0.000235,0.998717,0.999670,0.000801,0.995307,7.260155e-05,2.754768e-04,0.006306
5,2,False,cos,tanh,1210,666,145,NaN,NaN,NaN,0.999858,3.895113e-04,0.996904,0.999858,3.895113e-04,0.996904,0.992045,0.026662,0.827606,0.999430,0.001240,0.992697,1.937214e-07,5.564655e-07,0.000006
6,2,False,vonMises,max0x,7260,1480,747,NaN,NaN,NaN,0.995067,1.400187e-02,0.895580,0.995067,1.400187e-02,0.895580,0.999770,0.000480,0.993448,0.999555,0.001137,0.989141,7.031019e-05,2.533025e-04,0.007091
7,2,False,vonMises,tanh,7260,1216,342,NaN,NaN,NaN,0.999845,7.359128e-04,0.988839,0.999845,7.359128e-04,0.988839,0.992226,0.022194,0.827606,0.999514,0.001590,0.978071,2.643250e-06,5.203452e-05,0.002077


# Three rings

In [8]:
stat_3ring_eqw = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func equality', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: RL same $d_u-2k\omega$', 'SD: RL same $d_u-2k\omega$', 'Max: RL same $d_u-2k\omega$', 'Mean: RL Corr (A)', 'SD: RL Corr (A)', 'Min RL Corr (A)', 'Mean: RL Corr (B)', 'SD: RL Corr (B)', 'Min RL Corr (B)', 'Mean: RL Corr (C)', 'SD: RL Corr (C)', 'Min RL Corr (C)', 'Mean: RL Corr (D)', 'SD: RL Corr (D)', 'Min RL Corr (D)', 'Mean: Sym ring Sym $d_{rel}$', 'SD: Sym ring Sym $d_{rel}$', 'Max: Sym ring Sym $d_{rel}$'])

stat_3ring_neqw = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func equality', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: RL Corr (A)', 'SD: RL Corr (A)', 'Min RL Corr (A)', 'Mean: RL Corr (B)', 'SD: RL Corr (B)', 'Min RL Corr (B)', 'Mean: RL Corr (C)', 'SD: RL Corr (C)', 'Min RL Corr (C)', 'Mean: RL Corr (D)', 'SD: RL Corr (D)', 'Min RL Corr (D)',  'Mean: RL Sym $d_{rel}$', 'SD: RL Sym $d_{rel}$', 'Max: RL Sym $d_{rel}$', 'Mean: Sym ring Sym $d_{rel}$', 'SD: Sym ring Sym $d_{rel}$', 'Max: Sym ring Sym $d_{rel}$'])

structure = 'Shifter-ring (three rings)'
wfun_eqs = [True]*4 + [False]*4
for iorder, config in enumerate(tqdm(configs[12:20])):
    wfun_eq = wfun_eqs[iorder]

    net_stationary = load_gridsearch_results("stationary", config)
    net_moving = load_gridsearch_results("moving", config)
    net_moving.calculate_linear_range(net_stationary, config)
    
    # Theoretical prediction examination
    # Input - R-L correlation
    bump_amplitudes = cal_firate_a_acv_mean_a_peak(net_moving.activity, config.inputs, net_moving.part_linear_id, config.bs, config.actfun)
    input_diff_cors, input_diff_ps = cal_input_diff_cor(config.inputs, bump_amplitudes[4:], net_moving.part_linear_id, net_moving.linear_range_id) 
    show_value = input_diff_cors[net_moving.part_linear_id]
    
    # Calculate the mean, sd, min of the correlations
    cor_list = []
    for i in range(4):
        cor_list.append(np.nanmean(show_value[:,i]))
        cor_list.append(np.nanstd(show_value[:,i]))
        cor_list.append(np.nanmin(show_value[:,i]))
        
        cor_array_list[i].append(show_value[:,i])

    if wfun_eq: # If Weight fun equal
        # The sameness of the left and right rings
        index_shape_mismatch, dev_shape_ratios, if_match = cal_lr_shape_same_loop(net_moving.activity, net_stationary.valid_id, config.zeroid)
        dev_shape_ratios = dev_shape_ratios[net_stationary.valid_id]
        
        lr_same_pro, mean_same_dev, sd_same_dev, max_same_dev = 100-len(index_shape_mismatch)/len(net_stationary.valid_id) * 100, \
                np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios)
        rl_same_dev_list.append(dev_shape_ratios)
        
    
    else: # If Weight fun not equal
        # Mirror symmetry of the left and right rings
        index_shape_mismatch, dev_shape_ratios, if_match = cal_lr_shape_match_loop(net_moving.activity, net_moving.stable_id, config.zeroid)
        dev_shape_ratios = dev_shape_ratios[net_moving.stable_id]
        
        lr_match_pro, mean_lrmatch_dev, sd_lrmatch_dev, max_lrmatch_dev = 100-len(index_shape_mismatch)/len(net_moving.stable_id) * 100, \
            np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios)
        rl_sym_dev_list.append(dev_shape_ratios)

    # Mirror symmetry of the central ring
    index_shape_mismatch, dev_ratios, if_match = cal_central_shape_match_loop(net_moving.activity, net_moving.stable_id, config.zeroid)
    dev_ratios = dev_ratios[net_moving.stable_id]
    
    mean_cmatch_dev, sd_cmatch_dev, max_cmatch_dev = np.mean(dev_ratios), np.std(dev_ratios), np.max(dev_ratios)
    sym_sym_dev_list.append(dev_ratios)

    # Store the results
    linear_num = len(net_moving.linear_id)
    if wfun_eq:
        stat_3ring_eqw.loc[iorder] = [structure, config.ring_num, wfun_eq, waf_df_names[config.weight_fun.__name__], waf_df_names[config.actfun.__name__],total_num, net_stationary.valid_num, linear_num, mean_same_dev, sd_same_dev, max_same_dev] + cor_list + [mean_cmatch_dev, sd_cmatch_dev, max_cmatch_dev]
    else:
        stat_3ring_neqw.loc[iorder] = [structure, config.ring_num, wfun_eq, waf_df_names[config.weight_fun.__name__], waf_df_names[config.actfun.__name__], total_num, net_stationary.valid_num, linear_num] + cor_list + [mean_lrmatch_dev, sd_lrmatch_dev, max_lrmatch_dev, mean_cmatch_dev, sd_cmatch_dev, max_cmatch_dev]

stat_3ring = pd.concat([stat_3ring_eqw, stat_3ring_neqw], ignore_index=True)
stat_3ring.to_pickle(STAT_PATH / 'stat_3ring.npy')
stat_3ring.to_csv(TABLE_PATH / 'Supplementary table 3. Simulation results of shifter-ring network with three rings.csv')

 38%|███▊      | 3/8 [01:03<02:14, 26.98s/it]c:\ProgramData\miniforge3\envs\HD\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
 88%|████████▊ | 7/8 [02:08<00:17, 17.02s/it]c:\ProgramData\miniforge3\envs\HD\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
100%|██████████| 8/8 [02:25<00:00, 18.15s/it]


In [9]:
stat_3ring.iloc[:,1:]

,Ring No.,Weight Func equality,Weight Func,Act Func,N,N: valid stationary,N: linearly integrates,Mean: RL same $d_u-2k\omega$,SD: RL same $d_u-2k\omega$,Max: RL same $d_u-2k\omega$,Mean: RL Corr (A),SD: RL Corr (A),Min RL Corr (A),Mean: RL Corr (B),SD: RL Corr (B),Min RL Corr (B),Mean: RL Corr (C),SD: RL Corr (C),Min RL Corr (C),Mean: RL Corr (D),SD: RL Corr (D),Min RL Corr (D),Mean: Sym ring Sym $d_{rel}$,SD: Sym ring Sym $d_{rel}$,Max: Sym ring Sym $d_{rel}$,Mean: RL Sym $d_{rel}$,SD: RL Sym $d_{rel}$,Max: RL Sym $d_{rel}$
0,3,True,cos,max0x,7260,1100,240,3.842083e-17,2.616604e-16,1.210605e-14,1.000000,8.898410e-18,1.000000,1.000000,4.791940e-17,1.000000,1.000000,8.898410e-18,1.000000,0.999921,0.000160,0.998859,6.331700e-08,7.031264e-07,0.000036,NaN,NaN,NaN
1,3,True,cos,tanh,7260,953,258,5.918246e-17,5.498467e-16,7.905221e-14,1.000000,0.000000e+00,1.000000,1.000000,5.276814e-17,1.000000,0.988860,7.645153e-03,0.972758,0.999898,0.000139,0.998355,2.343789e-08,6.215433e-08,0.000002,NaN,NaN,NaN
2,3,True,vonMises,max0x,7260,6591,3335,4.223502e-17,4.838417e-16,6.652033e-14,1.000000,1.539184e-17,1.000000,1.000000,4.391803e-17,1.000000,0.999911,3.650727e-04,0.995957,0.999279,0.001512,0.990286,7.329334e-07,7.906595e-06,0.000523,NaN,NaN,NaN
3,3,True,vonMises,tanh,7260,5314,2449,4.904670e-17,3.311373e-16,6.291243e-14,1.000000,1.733645e-17,1.000000,1.000000,4.327439e-17,1.000000,0.994041,7.455554e-03,0.827606,0.998464,0.001836,0.988721,3.530131e-08,3.522008e-07,0.000052,NaN,NaN,NaN
4,3,False,cos,max0x,7260,1094,26,NaN,NaN,NaN,0.999658,5.304564e-04,0.995766,0.997597,1.044360e-02,0.906078,0.989607,4.230453e-02,0.689761,0.994985,0.019554,0.847545,7.888338e-07,1.227720e-05,0.000292,0.000006,0.000069,0.001206
5,3,False,cos,tanh,7260,772,12,NaN,NaN,NaN,0.998323,3.452016e-03,0.981755,0.997293,5.184346e-03,0.972816,0.955402,4.371581e-02,0.828759,0.996937,0.005068,0.973879,2.864094e-06,1.500703e-05,0.000203,0.000006,0.000021,0.000205
6,3,False,vonMises,max0x,7260,3358,1564,NaN,NaN,NaN,0.999841,4.469608e-04,0.995582,0.998752,1.150360e-02,0.809635,0.997183,2.840278e-02,0.565717,0.998383,0.014628,0.760062,4.272501e-07,1.020934e-05,0.001248,0.000001,0.000021,0.001647
7,3,False,vonMises,tanh,7260,2675,1075,NaN,NaN,NaN,0.999862,4.631367e-04,0.994178,0.999652,8.152086e-04,0.991595,0.943775,3.824584e-02,0.827606,0.999155,0.001393,0.984575,1.088490e-05,4.640932e-04,0.043419,0.000023,0.000694,0.051953


In [10]:
stat_shifter = pd.concat([stat_2ring, stat_3ring], ignore_index=True)
stat_shifter.loc[len(stat_shifter)] = [structure, config.ring_num, None, 'all', 'all', 
                                       stat_shifter['N'].sum(), 
                                       stat_shifter['N: valid stationary'].sum(), 
                                       stat_shifter['N: linearly integrates'].sum(), 
    np.nanmean(np.concatenate(rl_same_dev_list)), np.nanstd(np.concatenate(rl_same_dev_list)), np.nanmax(np.concatenate(rl_same_dev_list)), 
    np.nanmean(np.concatenate(cor_array_list[0])), np.nanstd(np.concatenate(cor_array_list[0])), np.nanmin(np.concatenate(cor_array_list[0])), 
    np.nanmean(np.concatenate(cor_array_list[1])), np.nanstd(np.concatenate(cor_array_list[1])), np.nanmin(np.concatenate(cor_array_list[1])), 
    np.nanmean(np.concatenate(cor_array_list[2])), np.nanstd(np.concatenate(cor_array_list[2])), np.nanmin(np.concatenate(cor_array_list[2])), 
    np.nanmean(np.concatenate(cor_array_list[3])), np.nanstd(np.concatenate(cor_array_list[3])), np.nanmin(np.concatenate(cor_array_list[3])), 
    np.nanmean(np.concatenate(rl_sym_dev_list)), np.nanstd(np.concatenate(rl_sym_dev_list)), np.nanmax(np.concatenate(rl_sym_dev_list)), 
    np.nanmean(np.concatenate(sym_sym_dev_list)), np.nanstd(np.concatenate(sym_sym_dev_list)), np.nanmax(np.concatenate(sym_sym_dev_list))]
stat_shifter.to_pickle(STAT_PATH / 'stat_shifter.npy')

In [11]:
stat_shifter.iloc[16:,1:]

,Ring No.,Weight Func equality,Weight Func,Act Func,N,N: valid stationary,N: linearly integrates,Mean: RL same $d_u-2k\omega$,SD: RL same $d_u-2k\omega$,Max: RL same $d_u-2k\omega$,Mean: RL Corr (A),SD: RL Corr (A),Min RL Corr (A),Mean: RL Corr (B),SD: RL Corr (B),Min RL Corr (B),Mean: RL Corr (C),SD: RL Corr (C),Min RL Corr (C),Mean: RL Corr (D),SD: RL Corr (D),Min RL Corr (D),Mean: RL Sym $d_{rel}$,SD: RL Sym $d_{rel}$,Max: RL Sym $d_{rel}$,Mean: Sym ring Sym $d_{rel}$,SD: Sym ring Sym $d_{rel}$,Max: Sym ring Sym $d_{rel}$
16,3,None,all,all,84484,29369,11001,1.319799e-16,1.538432e-15,2.165755e-13,0.99959,0.003963,0.89558,0.999402,0.005781,0.809635,0.993288,0.021467,0.565717,0.998825,0.008473,0.323207,0.000022,0.000379,0.051953,0.000002,0.000161,0.043419
